#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 모델링

## 0.미션

* 다음 세가지 챗봇을 만들고 비교해 봅시다.
* 챗봇1. Word2Vec 임베딩 벡터 기반 머신러닝 분류 모델링
    * Word2Vec 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
        * 이때, LightGBM을 추천하지만, 다른 알고리즘을 이용할수 있습니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.
* 챗봇2. 단계별 모델링1
    * 1단계 : type(일상대화 0, 에이블스쿨Q&A 1) 분류 모델 만들기
        * Embedding + LSTM 모델링
    * 2단계 : 사전학습된 Word2Vec 모델을 로딩하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기
* 챗봇3. 단계별 모델링2
    * 1단계 : 챗봇2의 1단계 모델을 그대로 활용
    * 2단계 : FastText 모델 생성하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기

* 챗봇3개에 대해서 몇가지 질문을 입력하고 각각의 답변을 비교해 봅시다.


## 1.환경준비

### (1)라이브러리 설치

#### 1) gensim 3.8.3 설치

In [1]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
# 현재 4.x 버전이 최신이지만, 3.8.3 버전으로 진행
!pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26528001 sha256=b02cbbcd9bab41edc56dd40d85310565c7f317154ed6926054592eb0346b285b
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


* [코랩] 위 라이브러리 설치후 런타임 재시작 필요!

#### 2) 형태소 분석을 위한 라이브러리

In [2]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.18).
git is already the newest version (1:2.25.1-1ubuntu3.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Buil

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.7 MB/s eta 0:00:00
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  2330k      0 --:--:-- --:--:-- --:--:-- 4935k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1


### (2) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 필요하다고 판단되는 라이브러리를 추가하세요.
import os

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import * 

import tensorflow as tf
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics.pairwise import cosine_similarity



* 형태소 분석을 위한 함수를 제공합니다.

In [5]:
# 1. 특수문자 제거
import re

def remove_special_chars(text):
    # 한글, 알파벳, 숫자를 제외한 모든 문자 제거
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\\s]", "", text)
    # 공백을 기준으로 단어 분리
    text = " ".join(text.split())
    return text

In [6]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [7]:
# 2. 형태소 분석기 사용
# 형태소 분석을 수행하는 함수

def tokenize(tokenizer_name, original_sent, nouns=False):
    # 미리 정의된 몇 가지 tokenizer 중 하나를 선택
    tokenizer = get_tokenizer(tokenizer_name)

    # 특수문자 제거
    sentence = remove_special_chars(original_sent)

    # tokenizer를 이용하여 original_sent를 토큰화하여 tokenized_sent에 저장하고, 이를 반환합니다.
    # sentence = original_sent.replace('\n', '').strip() # 줄바꿈 단위로 문장 판단 
    if nouns:       
        # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.nouns(sentence)
    else:
        tokens = tokenizer.morphs(sentence)
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent

### (3) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * train, test
    * 형태소분석 결과 데이터 : clean_train_questions, clean_test_questions

* 구글 드라이브 연결

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
path = '/content/drive/MyDrive/KT aivleschool/KT_aivle_school_AI_track/16. MINI_PJ6/'

* 저장된 .pkl 파일들을 불러옵니다.
* 불러 온 후에는 shape를 확인해 봅시다.

In [10]:
import joblib

train = joblib.load(path + 'train.pkl')
test = joblib.load(path + 'test.pkl')
clean_train_question = joblib.load(path + 'clean_train_questions.pkl')
clean_test_question = joblib.load(path + 'clean_test_questions.pkl')


In [11]:
print(train.shape)
print(test.shape)
print(clean_train_question.shape)
print(clean_test_question.shape)

(1156, 4)
(106, 4)
(1156,)
(106,)


In [12]:
train.tail()

,intent,Q,A,type
786,53,다른 국비지원과 비교할 때 차이가 뭔가요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
788,53,다른 교육과의 차이점은 무엇인가요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
789,53,다른 교육과 차이점이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
790,53,다른 국비 지원 교육보다 특별한 점이 있는지 궁금해요,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
791,53,다른 교육보다 특별한 것이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1


In [13]:
clean_train_question.tail()

786           다른 국비 지원 과 비교 할 때 차이 가 뭔 가요
788                다른 교육 과 의 차이점 은 무엇 인가요
789                    다른 교육 과 차이점 이 있 나요
790    다른 국비 지원 교육 보다 특별 한 점 이 있 는지 궁금 해요
791                다른 교육 보다 특별 한 것 이 있 나요
Name: Q, dtype: object

## 2.챗봇1

* **상세요구사항**
    * Word2Vec을 활용한 LightGBM 모델링(intent 분류)
        * Word2Vec을 이용하여 임베딩벡터 생성하기
            * Word Embedding으로 문장벡터 구하기
        * 임베딩 벡터를 이용하여 ML기반 모델링 수행하기
            * LightGBM 권장(다른 알고리즘을 이용할수 있습니다.)
    * 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
        * 질문을 입력받아, 답변하는 함수 생성

### (1) Word2Vec을 이용하여 임베딩벡터 생성하기
* 'mecab' 형태소 분석기를 이용하여 문장을 tokenize
    * Word2Vec 모델을 만들기 위해서 입력 데이터는 리스트 형태여야 합니다.
    * 그래서 다시 리스트로 저장되도록 토크나이즈 해 봅시다.
* Word Embedding으로 문장벡터를 생성합니다.
    * 먼저 Word2Vec 모델을 만들고, train의 질문들을 문장벡터로 만듭시다.


#### 1) 'mecab' 형태소 분석기를 이용하여 문장을 tokenize

In [14]:
tokenizer_names = ["mecab"]
for tokenizer_name in tokenizer_names:
    print(f"--- Tokenizer: {tokenizer_name} ---")
    for sent in train['Q']:
        tokenized_sent = tokenize(tokenizer_name, sent)
        print(f"원본 문장: {sent}")
        print(f"토큰화된 문장: {tokenized_sent}")
    print("\n")

--- Tokenizer: mecab ---
원본 문장: 떨어뜨려서 핸드폰 액정 나갔어
토큰화된 문장: 떨어뜨려서 핸드폰 액정 나갔 어
원본 문장: 액정 나갔어
토큰화된 문장: 액정 나갔 어
원본 문장: 핸드폰 떨어뜨려서 고장 났나봐
토큰화된 문장: 핸드폰 떨어뜨려서 고장 났 나 봐
원본 문장: 노트북 키보드가 안먹히네
토큰화된 문장: 노트북 키보드 가 안 먹히 네
원본 문장: 노트북 전원이 안들어와.
토큰화된 문장: 노트북 전원 이 안 들어와
원본 문장: 컴퓨터가 제대로 작동하지 않아요
토큰화된 문장: 컴퓨터 가 제대로 작동 하 지 않 아요
원본 문장: 컴퓨터가 에러 메시지를 띄우고 있어요.
토큰화된 문장: 컴퓨터 가 에러 메시지 를 띄우 고 있 어요
원본 문장: 컴터 고장났나봐
토큰화된 문장: 컴 터 고장났 나 봐
원본 문장: 컴터가 맛이 갔어
토큰화된 문장: 컴 터 가 맛 이 갔 어
원본 문장: 컴터가 안돼
토큰화된 문장: 컴 터 가 안 돼
원본 문장: 핸드폰 고장났나봐
토큰화된 문장: 핸드폰 고장났 나 봐
원본 문장: 핸드폰 맛이 갔어
토큰화된 문장: 핸드폰 맛 이 갔 어
원본 문장: 핸드폰 안돼
토큰화된 문장: 핸드폰 안 돼
원본 문장: 노트북 안돼
토큰화된 문장: 노트북 안 돼
원본 문장: 내 의지는 상관없나봐
토큰화된 문장: 내 의지 는 상관없 나 봐
원본 문장: 내 의지로 안되는 일인가봐
토큰화된 문장: 내 의 지로 안 되 는 일 인가 봐
원본 문장: 건강이 최고
토큰화된 문장: 건강 이 최고
원본 문장: 건강이 최고인 것 같아
토큰화된 문장: 건강 이 최고 인 것 같 아
원본 문장: 아프면 안 되는데
토큰화된 문장: 아프 면 안 되 는데
원본 문장: 올해도 건강하길
토큰화된 문장: 올해 도 건강 하 길
원본 문장: 올해도 행복하길
토큰화된 문장: 올해 도 행복 하 길
원본 문장: 올해 합격하길
토큰화된 문장: 올해 합격 하 길
원본 문장: 건강이 최고야
토큰화된 문장: 건강 이 최고 야
원본 문장: 이번년도는 꼭 집을 장만할거야
토큰화된 문장:

In [15]:
train_tokenize = train["Q"].apply(lambda x: tokenize('mecab', x))

#### 2) Word Embedding으로 문장벡터 구하기
* Word2Vec
    * 위에서 저장한 입력 데이터를 사용하여 Word2Vec 모델이 생성
    * 모델은 size(단어 벡터의 차원), 
    * window(컨텍스트 창의 크기), 
    * max_vocab_size(고려할 최대 어휘 크기), 
    * min_count(포함할 단어의 최소 빈도)와 같은 특정 하이퍼파라미터로 훈련됩니다.
    * sg : 사용할 훈련 알고리즘 - 1은 skip-gram, 0은 CBOW )

In [16]:
import gensim
from gensim.models import Word2Vec
import multiprocessing

# 문장 리스트를 토큰화하여 모델 입력 데이터로 변환
tokenized_sentences = [tokenize('mecab', sentence) for sentence in train['Q']]

# Word2Vec 모델 생성 및 학습
w2v_model = Word2Vec(
    sentences=tokenized_sentences,
    size=100, # 임베딩 벡터의 차원
    window=5, # 컨텍스트 창의 크기
    min_count=5, # 최소 단어 빈도
    sg=1, # 훈련 알고리즘 (1: skip-gram, 0: CBOW)
    workers=multiprocessing.cpu_count(),
    iter=50 # Epoch 수
)

# 학습된 모델 저장
w2v_model.save('w2v_model.model')


In [17]:
# from gensim.models import Word2Vec

# # Word2Vec 모델 생성
# wv_model = Word2Vec(                   )

* Word2Vec 모델로부터 데이터를 벡터화하기 위한 함수 생성

In [18]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.index2word)
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [19]:
# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)
    
    return sent_embedding_vectors

* 이제 학습데이터의 Q를 Word2Vec 모델을 사용하여 벡터화 합니다.

In [20]:
# 학습 데이터의 문장들을 Word2Vec 모델을 사용하여 벡터화
train_data_vecs = get_dataset(train['Q'], w2v_model, 100)

# 인자 설명 
# 1 - 학습데이터에서 질문에 해당하는 열
# 2 - word2vec 모델 객체
# 3- word2vec 모델에서 지정한 임베딩 벡터의 크기
# 이 코드를 실행하면 모든 질문들이 Word2Vec 모델을 사용하여 벡터화된 후 2차원 np 배열로 저장됨
# train_data_vecs = get_dataset(data['Q'], model, embedding_size)

In [21]:
train_data_vecs[:3]

array([[ 6.11033551e-02, -5.86793646e-02,  1.33948267e-01,
        -4.77688760e-02, -7.21168220e-02, -1.33244589e-01,
         2.71039128e-01, -2.39213184e-02,  2.99397141e-01,
        -2.69033574e-02, -1.18992500e-01, -3.12213480e-01,
         2.13366255e-01, -2.65664637e-01,  1.15424059e-01,
         3.91899608e-02,  1.10909000e-01,  4.62106569e-03,
        -1.17001221e-01,  2.83167243e-01, -1.39235765e-01,
        -3.57486278e-01, -4.36451852e-01,  2.88090259e-01,
         2.07138974e-02,  2.02885017e-01,  6.54323306e-03,
         1.55535951e-01,  1.00249453e-02, -1.33970752e-01,
         1.95604917e-02,  1.40405148e-01,  6.20643608e-02,
        -2.27081534e-02, -1.22188896e-01, -4.52985376e-04,
        -6.59853444e-02, -4.57786508e-02,  1.79839611e-01,
        -1.51610315e-01, -1.80651024e-01, -6.83746263e-02,
         2.87280470e-01, -1.56863883e-01,  1.23679370e-01,
         9.91691127e-02, -1.15185166e-02,  1.42001929e-02,
        -4.92943190e-02, -2.55953282e-01,  1.94356795e-0

* 훈련된 Word2Vec 모델을 사용하여 문장 목록에 대한 문장 임베딩을 생성하고 이를 2차원 numpy 배열에 저장합니다. 
* 그런 다음 이러한 임베딩을 다양한 기계 학습 모델의 입력 기능으로 사용할 수 있습니다

### (2) 분류 모델링
* 데이터 분할
    * x, y
        * x : 이전 단계에서 저장된 임베딩벡터(train_data_vecs)
        * y : intent 값들
    * train, val
        * train_test_split 활용
* 머신러닝 모델링
    * lightGBM, RandomForest 등을 활용하여 학습
    * 필요하다면 hyper parameter 튜닝을 시도해도 좋습니다.
* validation set으로 검증해 봅시다.

In [22]:
# X와 y 데이터 분리
X = train_data_vecs
y = train['intent']

# Train-Test split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

* 모델1

In [23]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# LightGBM 모델 생성 및 학습
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

# RandomForest 모델 생성 및 학습
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# 검증셋으로 성능 측정
y_pred_lgbm = lgbm.predict(X_val)
y_pred_rf = rf.predict(X_val)
print('LightGBM accuracy: ', accuracy_score(y_val, y_pred_lgbm))
print('RandomForest accuracy: ', accuracy_score(y_val, y_pred_rf))


LightGBM accuracy:  0.6637931034482759
RandomForest accuracy:  0.6379310344827587


* 모델2

### (3) 챗봇 구축

* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 데이터 중 하나에 대해서 테스트

In [28]:
# answer_dict = train.groupby('intent')['A'].apply(list).to_dict()
answer_dict = {}

for intent, group in train.groupby("intent"):
    unique_answers = group["A"].unique()  # 중복 제거
    answer_dict[intent] = list(unique_answers)  # answer_dict에 추가


In [29]:
answer_dict

{1: ['as 맡기세요.', 'AS센터에 맡겨보세요.'],
 2: ['가장 중요한 거예요.', '가장 중요한 목표네요.'],
 3: ['감기 조심하세요.'],
 4: ['같이 가보세요.', '같이 가요.', '같이 가자고 말해보세요'],
 5: ['고민은 누구나 해요.', '고민이 많은가 봅니다.', '고민이 많은가 봐요.', '고민이 있나 봐요.', '고민이 있나봐요.'],
 6: ['공부 좋죠.', '공부가 최우선이죠.', '공부는 끝이 없죠.', '공부는 뭐든 좋아요.', '공부는 언제나 좋죠.'],
 7: ['그런 친구 말고 제대로 된 친구 사귀세요.', '그런 친구는 거르세요.'],
 8: ['기분전환을 해보세요.',
  '힘들 때네요.',
  '힘들겠네요.',
  '힘들겠어요.',
  '너무 무리하면 지쳐요.',
  '너무 무리하지 마세요.',
  '너무 무리하지는 마세요.',
  '너무 신경 곤두세우지 마세요.',
  '너무 신경쓰지 말고 그러든지 하고 아무렇지도 않게 넘겨보세요.',
  '너무 신경쓰지마세요.',
  '많이 지쳤나봐요.',
  '많이 지치고 힘든가봅니다.',
  '많이 피곤한가봐요.',
  '직장 스트레스 심하겠네요.',
  '직장 스트레스가 심한가봐요.'],
 9: ['꾸준히 약 먹고 치료해보세요.', '꾸준히 치료하세요.'],
 10: ['내 집 마련 축하드려요.'],
 11: ['냉장고 파먹기 해보세요.'],
 12: ['노력하겠습니다.', '노력하고 있는 중입니다.', '노력하고 있어요.'],
 13: ['다 잘 될 거예요.',
  '다 지나갈 거예요.',
  '성공을 기원합니다.',
  '성공을 빌어요.',
  '성공적으로 끝날 거예요.',
  '성공적으로 마치길 바랄게요.',
  '성공하길 바랍니다.',
  '성공했길 바랍니다.'],
 14: ['다녀오세요!', '다녀오세요. 기다릴게요.'],
 15: ['대중교통을 이용해 보세요.', '대중교통을 이용해주세요.'],
 16: ['동감이에요.', '동감입니다.'],
 

In [39]:
import random

def chatbot(model, answer_dict, tokenizer, w2v_model, num_features, question):
    # 입력 받은 질문을 형태소 분석합니다.
    question = remove_special_chars(question)
    tokens = tokenize(tokenizer, question)
    
    # 문장을 Word2Vec 모델로부터 벡터화합니다.
    sentence_vec = get_sent_embedding(w2v_model, num_features, tokens).reshape(1, -1)

    # 분류 모델을 사용하여 입력 받은 질문의 의도를 예측합니다.
    pred = model.predict(sentence_vec)[0]
    
    # 분류 모델의 예측 결과에 해당하는 답변을 랜덤하게 선택합니다.
    answer_list = answer_dict[pred]
    answer = random.choice(answer_list)
    
    return answer


In [42]:
# while True:
#     question = input("질문을 입력하세요: ")
#     # question_vec = vectorizer.transform([question_text]).reshape(1, -1) # 2차원 배열로 변환
#     # answer = chatbot(model, answer_dict, tokenizer, question)
#     answer = chatbot(lgbm, answer_dict, 'mecab', question)
#     print(answer)
for i in range(5):
    question = input("질문을 입력하세요: ")
    answer = chatbot(lgbm, answer_dict, 'mecab', w2v_model, 100, question)
    print(answer)


질문을 입력하세요: 배고파
다 지나갈 거예요.
질문을 입력하세요: 배고프다니까
가장 중요한 목표네요.
질문을 입력하세요: 뭐해; 배고프다고
힘들겠어요.
질문을 입력하세요: 그래 나 배고프고 힘들어
저도요.
질문을 입력하세요: ㅋㅋㅋㅋㅋ 응...
동감이에요.


AttributeError: ignored

#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용. 

In [ ]:
def get_answer1(question): 




    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 3.챗봇2

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : type을 0과 1로 분류하는 모델 생성(Embedding + LSTM 모델)
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링(LSTM)
- LSTM

#### 1) 데이터 준비
* 학습용 데이터를 만들어 봅시다.
    * 시작 데이터 : clean_train_questions, clean_test_questions
    * 각 토큰에 인덱스를 부여하는 토크나이저를 만들고 적용
        * from tensorflow.keras.preprocessing.text import Tokenizer 를 사용
    * 문장별 길이에 대한 분포를 확인하고 적절하게 정의.

In [ ]:
# 각각의 토큰에 인덱스 부여하는 토크나이저 선언


# .fit_on_tests 이용하여 토크나이저 만들기



In [ ]:
# 전체 토큰의 수 확인



In [ ]:
# 전체 토큰의 수로 vocab_size 지정


# fit_on_texts을 위에서 한번만 해도 되지만, vocab 사이즈를 확인하고 줄이거나 하는 시도를 할 수도 있기에 다시 수행


# .texts_to_sequences : 토크나이즈 된 데이터를 가지고 모두 시퀀스로 변환
x_train =
x_val = 

In [ ]:
# 각 토큰과 인덱스로 구성된 딕셔너리 생성,


# <PAD> 는 0으로 추가



* 문장별 토큰수에 대해 탐색적 분석을 수행해 봅시다.

* 문장별 토큰이 가장 큰 것이 57개 입니다. 

* 학습 입력을 위한 데이터 크기 맞추기
    * 문장이 짧기 때문에 MAX_SEQUENCE_LENGTH는 정하지 않아도 되지만,
    * 그러나 분포를 보고 적절하게 자릅시다.
    * 그리고 pad_sequences 함수를 이용하여 시퀀스데이터로 변환하기
* y는 train['type'] 와 test['type'] 입니다.

#### 2) 모델링

* 토크나이징 한 데이터를 입력으로 받아 
* Embedding 레이어와 LSTM 레이어를 결합하여 이진 분류 모델링을 수행합니다.

### (2) 사전학습모델(Word2Vec)

* Pre-trained Word2Vec model
    * 이미 다운로드 받아서 제공되었습니다.
        * ko.bin, ko.tsv
    * 참고 사이트: https://github.com/Kyubyong/wordvectors
        * 모델 파일 다운로드 사이트: https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view?resourcekey=0-Dq9yyzwZxAqT3J02qvnFwg
* 사전학습 모델을 로딩하고, 
* train 데이터셋의 질문(Q)을 임베딩벡터로 만들어, 열(Column)로 추가합니다.

#### 1) 모델 로딩
* 사전 학습된 모델을 로딩 : gensim.models.Word2Vec.load()
* 로딩 후 벡터 크기를 조회합시다. .vector_size

In [ ]:
import gensim
pre_wv_model = gensim.models.Word2Vec.load(path + 'ko.bin')

In [ ]:
# 모델의 벡터크기 조회



#### 2) train 에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

### (3) 챗봇 구축
* 아래 절차대로 수행하는 함수 만들기
    * input 질문 
    * 1단계 : 모델을 이용하여 type 0, 1로 분류
    * 2단계 : 
        * train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
        * 코사인 유사도가 가장 높은 Q를 선택
        * 선택한 Q의 intent에 맵핑된 답변 중 하나를 무작위로 선택

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

* 예측을 위한 입력 형태로 변환
    * 학습을 위한 전처리 과정을 test 데이터에도 적용합니다. 

* 1단계 : type 분류

* 2단계 : 질문에 대한 벡터 만들고 코사인 유사도 계산
    * Word2Vec 사전 학습 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * Word2Vec 으로 만든 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity






#### 2) 챗봇 함수 만들기
* 위 테스트 결과를 바탕으로 코드를 정리하고 함수로 생성합니다.

In [ ]:
def get_answer2(question): 





    return

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 4.챗봇3

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : 챗봇2의 1단계 모델을 사용합니다.
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링
- LSTM : 3-(1) 모델을 그대로 사용합니다.

### (2) FastText 모델

-  FastText 모델 학습을 위한 입력 포맷 2차원 리스트 형태 입니다.
  ```
  [['나', '는', '학생', '이다'], ['오늘', '은', '날씨', '가', '좋다']]
  ```

- Word2Vec계열의 FastText를 학습하는 이유
  - n-gram이 추가된 fasttext 모델은 유사한 단어에 대한 임베딩을 word2vec보다 잘 해결할 수 있으며, 오탈자 등에 대한 임베딩 처리가 가능하다.
  - 예) 체크카드, 쳌카드는 word2vec에서는 전혀 다른 단어이지만 fasttext는 character n-gram으로 비교적 같은 단어로 처리할 수 있다.
- 참고: https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText


#### 1) 데이터 준비
* 시작데이터 : clean_train_questions, clean_test_questions

* FastText를 위한 입력 데이터 구조 만들기

#### 2) FastText 모델 생성
* FastText 문법
    * FastText( input데이터,  min_count = , size= , window=  )
        * input데이터 : 학습에 사용할 문장으로 이루어진 리스트
        * min_count : 모델에 사용할 단어의 최소 빈도수. 이 값보다 적게 출현한 단어는 모델에 포함되지 않음. 기본값 = 5
        * size : 단어의 벡터 차원 지정. 기본값 = 100
        * window : 학습할 때 한 단어의 좌우 몇 개의 단어를 보고 예측을 할 것인지를 지정. 기본값 = 5
    * 참조 : https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText

In [ ]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec



#### 3) train에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

### (3) 챗봇 구축
- input 질문
- intent classifier로 common와 faq 중 하나를 예측
- 예측된 intent에 속한 train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
- 코사인 유사도가 가장 높은 top-3개의 Q를 선택
- 선택한 Q에 맵핑된 답변 중 하나를 선택하고 실제 답변과 비교

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

* 예측을 위한 입력 형태로 변환

* 예측하기

* 질문에 대한 벡터 만들기
    * FestText 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * FastText 로 만들 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity






#### 2) 함수로 생성하기

In [ ]:
def get_answer3(question): 







    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 5.질문에 대한 답변 비교해보기

* **세부요구사항**
    * 세가지 챗봇을 생성해 보았습니다. 
    * 질문을 입력하여 답변을 비교해 봅시다. 어떤 챗봇이 좀 더 정확한 답변을 하나요?
